# Advanced Python: Building Scalable Applications

### Day 5: Serialization, Data Persistence and Web APIs


In [2]:
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.shutdown?

Docstring:
shutdown(flag)

Shut down the reading side of the socket (flag == SHUT_RD), the writing side
of the socket (flag == SHUT_WR), or both ends (flag == SHUT_RDWR).
Type:      builtin_function_or_method
